Ce notebook permet de voir la différence entre l'autonomie réelle et l'autonomie annoncée par Tesla.
Et donc de voir si cette autonomie diminue en fonction du SoH 

Conclusion:
- L'autonomie réelle (en hiver) equivaut à environ 67% de l'autonomie annoncée
- l'autonomie réelle diminue en fonction du SoH pour 20% de SoH perdu la perte d'autonomie réelle est en moyenne de 25%. 

L'étude a été faite sur les tesla de chez Ayvens qu'on a observé durant la période hivernale.


In [ ]:

import pandas as pd
from core.pandas_utils import *
from transform.processed_phases.config import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from core.s3.s3_utils import S3Service, S3Settings
from core.spark_utils import create_spark_session


In [ ]:
settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3 = S3Service()

In [ ]:
from core.sql_utils import *
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join oem o
            on o.id=vm.oem_id
            AND o.oem_name = 'tesla';"""), con)

dbeaver_df.columns

In [ ]:
df = s3.read_parquet_df_spark(spark, 'processed_phases/processed_phases_tesla_fleet_telemetry.parquet').toPandas()
# result_phases = s3.read_parquet_df_spark(spark, 'result_phases/result_phases_tesla_fleet_telemetry.parquet').toPandas()

In [ ]:
df_merge_discharge = pd.merge_asof(df.sort_values('DATETIME_BEGIN'), dbeaver_df.sort_values('timestamp')[['timestamp', 'vin','soh', 'autonomy', 'model_name', 'type', 'version']], 
                                   left_on=['DATETIME_BEGIN'], right_on=['timestamp'], left_by='VIN', right_by='vin', direction='forward')

In [ ]:
# drop MT336 pour éviter tout problème
df_merge_discharge_no_mt336 = df_merge_discharge[df_merge_discharge['version']!='MT336']

In [ ]:
df_merge_discharge_no_mt336['ODOMETER_DIFF'] = df_merge_discharge_no_mt336['ODOMETER_LAST'] - df_merge_discharge_no_mt336['ODOMETER_FIRST']
df_merge_discharge_no_mt336['ratio_km_soc'] = df_merge_discharge_no_mt336['ODOMETER_DIFF'] / df_merge_discharge_no_mt336['SOC_DIFF'].abs() 
df_merge_discharge_no_mt336['ratio_km_soc_autonomy'] = 100 * df_merge_discharge_no_mt336['ratio_km_soc'] /  df_merge_discharge_no_mt336['autonomy']

In [ ]:
px.imshow(df_merge_discharge_no_mt336.select_dtypes(['int', 'float']).corr()[['ratio_km_soc', 'soh']].round(2), text_auto=True, width=600, height=800)

Le SoH ne semble pas corrélé direcetement avec le ratio_km_soc. Le soh n'est pas corréllé avec le nombre de km parcourus pendant la décharge. 

Le ratio et donc le nombre de km parcouru sur une décharge dépend majoritairemement de facteurs externes a la voiture:
- type de conduite (acceleration)
- parcours (dénivelé)
- condition ext (météo, vent ...)


## Study for one kind of vehicule

on récupère les modèles 3 rwd

In [ ]:
df_merge_discharge_no_mt336['autonomy'].unique()

In [ ]:
model_3_rwd = df_merge_discharge_no_mt336[(df_merge_discharge_no_mt336['model_name']=="model 3") & (df_merge_discharge_no_mt336['type']=="rwd") ]

In [ ]:
model_3_rwd = (model_3_rwd[['vin', 'SOC_DIFF', 'ODOMETER_DIFF', 'SOC_FIRST', 'SOC_LAST', 'soh', 'autonomy', 'ratio_km_soc']]
               [(model_3_rwd['SOC_DIFF'] < -1) & (model_3_rwd['ODOMETER_DIFF'] > 0)])

In [ ]:
med_ratio = model_3_rwd.groupby('vin')[["ratio_km_soc", "soh"]].median()

In [ ]:
px.histogram(model_3_rwd[model_3_rwd['vin']=="LRW3E7FSXRC069268"], x='ratio_km_soc', nbins=100, title="répartition du ratios Km parcouru / nombre SoC points perdu pour le vin LRW3E7FSXRC069268",)

In [ ]:
px.scatter(med_ratio, x='soh', y='ratio_km_soc',  
           title="distribution des ratio median par rapport au SoH des vin pour des tesla model 3 rwd", 
           )

## Graph on all vehicules

In [ ]:
# on utilise la mediane ici pour éviter d'être biaisé par des valeurs trop élevées/faibles de ratio 
gb_vin = df_merge_discharge_no_mt336[df_merge_discharge_no_mt336['soh'] > .83].groupby(['VIN', 'model_name', 'type', 'BATTERY_NET_CAPACITY'], observed=True, as_index=False)[['soh', 'ratio_km_soc', 'ratio_km_soc_autonomy']].median()

In [ ]:
gb_vin.VIN.nunique()

In [ ]:
# on veut éviter les outliers 
gb_vin = gb_vin.dropna()[(gb_vin["ratio_km_soc"] < 7) & (gb_vin["ratio_km_soc"] > 1) ]

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True).fit(gb_vin[['soh']], gb_vin['ratio_km_soc_autonomy'])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gb_vin['soh'], y=gb_vin["ratio_km_soc_autonomy"], mode='markers', name="Points"))

fig.add_trace(go.Scatter(x=np.array([0.8, 1]), 
                         y=model.predict(pd.DataFrame({'soh':[.8, 1]})), 
                         mode='lines', name="Ligne", line=dict(color='red')))

fig.show()

In [ ]:

model.predict(pd.DataFrame({'soh':[.8]})) / model.predict(pd.DataFrame({'soh':[1]}))

On constate une dégradation moyenne de 27.1% pour l'autonomie pour une dégradation de supposé de 20% pour le SoH en prenant l'ensemble des véhicules.  
Ce qui signifie que lorsque le soh se dégrade de 1 point on perd en moyenne 1.04 % d'autonomie. 

In [ ]:
gb_vin.columns

In [ ]:

# Filtrer les données
filtered_data = gb_vin[(gb_vin['ratio_km_soc_autonomy'] > 0) & 
                        (gb_vin['ratio_km_soc_autonomy'] < 3)][['soh', 'ratio_km_soc_autonomy', 'model_name']].dropna()

# Créer un histogramme (barplot)
px.histogram(filtered_data, 
             x='ratio_km_soc_autonomy', 
             color='model_name',  
             title='Répartition autonomie réelle',
             opacity=.5)



In [ ]:

fig = ff.create_distplot([gb_vin[ 
                                             (gb_vin['ratio_km_soc_autonomy'] > 0) & 
                                             (gb_vin['ratio_km_soc_autonomy'] < 3)][['soh', 'ratio_km_soc_autonomy']].dropna()['ratio_km_soc_autonomy'].values],
                         ['ratio_km_soc_autonomy'], 
                         colors= ['green'],
                         bin_size=.01,
             
             )
fig.show()

## decroissance par model de vehicules

In [ ]:
gb_vin.model_name.unique()

In [ ]:
sub_set = gb_vin[(gb_vin['model_name']=='model 3') & (gb_vin['type']=='rwd')]

In [ ]:
model = LinearRegression(fit_intercept=True).fit(sub_set[['soh']], sub_set['ratio_km_soc_autonomy'])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sub_set['soh'], y=sub_set["ratio_km_soc_autonomy"], mode='markers'))

fig.add_trace(go.Scatter(x=np.array([0.8, 1]), 
                         y=model.predict(pd.DataFrame({'soh':[.8, 1]})), 
                         mode='lines',  line=dict(color='red')))
fig.update_layout(title=dict(text='Relation entre SoH et le ratio km_soc/autonomy'))
fig.update_xaxes(title='SoH')
fig.update_yaxes(title='km/soc/autonomy')
fig.update_traces(showlegend=False)
fig.show()

In [ ]:
model.predict(pd.DataFrame({'soh':[.8]})) / model.predict(pd.DataFrame({'soh':[1]}))